# Notebook RPP News

### Importación de librerías

In [1]:
pip install wget

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install unicodecsv

Note: you may need to restart the kernel to use updated packages.


In [4]:
from bs4 import BeautifulSoup
import base64
import wget
import requests
import unicodecsv as csv
import datetime
import pytz
import dateutil.parser
import pandas as pd
import time

### Creación de arreglo de días

In [28]:
## get days
iterator_day = datetime.date(2019, 1, 1)
last_publication = datetime.date(2018, 9, 1)
array_days = [str(iterator_day)]
while(iterator_day >= last_publication):
    iterator_day = iterator_day - datetime.timedelta(days=1)
    array_days.append(str(iterator_day))

In [30]:
array_days

['2019-01-01',
 '2018-12-31',
 '2018-12-30',
 '2018-12-29',
 '2018-12-28',
 '2018-12-27',
 '2018-12-26',
 '2018-12-25',
 '2018-12-24',
 '2018-12-23',
 '2018-12-22',
 '2018-12-21',
 '2018-12-20',
 '2018-12-19',
 '2018-12-18',
 '2018-12-17',
 '2018-12-16',
 '2018-12-15',
 '2018-12-14',
 '2018-12-13',
 '2018-12-12',
 '2018-12-11',
 '2018-12-10',
 '2018-12-09',
 '2018-12-08',
 '2018-12-07',
 '2018-12-06',
 '2018-12-05',
 '2018-12-04',
 '2018-12-03',
 '2018-12-02',
 '2018-12-01',
 '2018-11-30',
 '2018-11-29',
 '2018-11-28',
 '2018-11-27',
 '2018-11-26',
 '2018-11-25',
 '2018-11-24',
 '2018-11-23',
 '2018-11-22',
 '2018-11-21',
 '2018-11-20',
 '2018-11-19',
 '2018-11-18',
 '2018-11-17',
 '2018-11-16',
 '2018-11-15',
 '2018-11-14',
 '2018-11-13',
 '2018-11-12',
 '2018-11-11',
 '2018-11-10',
 '2018-11-09',
 '2018-11-08',
 '2018-11-07',
 '2018-11-06',
 '2018-11-05',
 '2018-11-04',
 '2018-11-03',
 '2018-11-02',
 '2018-11-01',
 '2018-10-31',
 '2018-10-30',
 '2018-10-29',
 '2018-10-28',
 '2018-10-

### Funciones

In [16]:
#Obtener links de archivo por días
def get_news_rpp(days):  
    links = []
    for day in days:           
        url_day = 'https://rpp.pe/archivo/'+day  
        content = requests.get(url_day).content
        soup_article = BeautifulSoup(content,'lxml')
        print("URL Day===>" + str(url_day))
        articles_day = soup_article.findAll(['article',{'class': 'card'}])
        for article in articles_day:            
            if len(article["class"]) == 1:                       
                if article.find('h3') is not None:
                    if article.find('h3').find('a') is not None:
                        tt = article.find('h3').find('a')["href"]
                        links.append(tt)
    return links

In [17]:
#Obtener el título del artículo
def get_title_rpp(soup_article):
    if soup_article.find("h1",{"itemprop":"headline"}) is not None:
        title = soup_article.find("h1",{"itemprop":"headline"}).text
    else:
        title = "None"
    return title

In [18]:
#Obtener la fecha del artículo
def get_date_rpp(soup_article):
    if soup_article.find("time",{"itemprop":"datePublished"}) is not None:
        fecha = soup_article.find("time",{"itemprop":"datePublished"})
        fecha_return = dateutil.parser.parse(fecha["datetime"])      
    else:
        fecha_return = "None"
    return fecha_return
    

In [19]:
#Obtener categoría
def get_cat_rpp(soup_article):
    if soup_article.find("span",{"itemprop":"name"}) is not None:
        categoria = soup_article.find("span",{"itemprop":"name"}).text
    else:
        categoria = "None"
    return categoria

In [20]:
#Obtener autor
def get_author_rpp(soup_article):
    if soup_article.find("div",{"id":"author-name"}) is not None:
        autor = soup_article.find("div",{"id":"author-name"}).text
    else:
        autor = "None"
    return autor

In [21]:
#Obtener resumen
def get_summary_rpp(soup_article):
    if soup_article.find("div",{"class":"sumary"}) is not None:
        resumen = soup_article.find("div",{"class":"sumary"})
        nonBreakSpace = u'\xa0'
        resumen_return = resumen.text.replace(nonBreakSpace, ' ')
    else:
        resumen_return = "None"
    return resumen_return

In [22]:
#Obtener contenido
def get_content_rpp(soup_article):
    if soup_article.find("div",{"id":"article-body"}) is not None:
        contenido_p = soup_article.find("div",{"id":"article-body"}).findAll(["p","h3"])
        content_line = []
        for cont_p in contenido_p:
            if cont_p.parent.has_attr('class'):
                try:
                    if cont_p.parent.name == "div" and cont_p.parent.attrs['class'][0] == "col-detail":
                        tt = cont_p.text
                        content_line.append(tt+" ")
                except IndexError:
                    pass
                continue       
        contenido = ''.join(content_line)
        nonBreakSpace = u'\xa0'
        contenido_return = contenido.replace(nonBreakSpace, ' ')        
    else:
        contenido_return = "None"
    return contenido_return

In [31]:
#Función principal
columns = ["Fecha", "Categoria", "Autor", "Titulo", "Resumen", "Contenido"]
d = []
df_RPP = []
def iterate_articles_rpp(days):
    links = get_news_rpp(days)   
    i = 0
    for link in links:        
        url_article = 'https://rpp.pe' + link
        content_article = requests.get(url_article).content
        soup_article = BeautifulSoup(content_article,'lxml')
        print("URL Article===>" + str(url_article))
        
        date = get_date_rpp(soup_article)     
        category = get_cat_rpp(soup_article)
        author = get_author_rpp(soup_article)
        title = get_title_rpp(soup_article)
        summary = get_summary_rpp(soup_article)
        content = get_content_rpp(soup_article)
        
        d.append((date, category, author, title, summary, content))
        i = i+1  
        print('-------------------- End Article ----------------------')    
    df = pd.DataFrame(d,columns=columns)
    return df

In [32]:
start = time.time()
df_RPP = iterate_articles_rpp(array_days)
end =time.time()
te = end-start
print("Tiempo de ejecución es %f segundos" %(te))

URL Day===>https://rpp.pe/archivo/2019-01-01
URL Day===>https://rpp.pe/archivo/2018-12-31
URL Day===>https://rpp.pe/archivo/2018-12-30
URL Day===>https://rpp.pe/archivo/2018-12-29
URL Day===>https://rpp.pe/archivo/2018-12-28
URL Day===>https://rpp.pe/archivo/2018-12-27
URL Day===>https://rpp.pe/archivo/2018-12-26
URL Day===>https://rpp.pe/archivo/2018-12-25
URL Day===>https://rpp.pe/archivo/2018-12-24
URL Day===>https://rpp.pe/archivo/2018-12-23
URL Day===>https://rpp.pe/archivo/2018-12-22
URL Day===>https://rpp.pe/archivo/2018-12-21
URL Day===>https://rpp.pe/archivo/2018-12-20
URL Day===>https://rpp.pe/archivo/2018-12-19
URL Day===>https://rpp.pe/archivo/2018-12-18
URL Day===>https://rpp.pe/archivo/2018-12-17
URL Day===>https://rpp.pe/archivo/2018-12-16
URL Day===>https://rpp.pe/archivo/2018-12-15
URL Day===>https://rpp.pe/archivo/2018-12-14
URL Day===>https://rpp.pe/archivo/2018-12-13
URL Day===>https://rpp.pe/archivo/2018-12-12
URL Day===>https://rpp.pe/archivo/2018-12-11
URL Day===

URL Article===>https://rpp.pe/politica/judiciales/concepcion-carhuancho-sobre-salida-de-fiscales-esto-confirma-la-captura-del-ministerio-publico-noticia-1172725
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/futbol/descentralizado/alianza-lima-se-cumplen-19-anos-del-fallecimiento-de-sandro-baylon-noticia-1172723
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/economia/economia/la-economia-peruana-cierra-el-2018-con-una-inflacion-del-248-noticia-1172720
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/mundo/latinoamerica/jair-bolsonaro-le-dice-a-donald-trump-que-juntos-le-daran-prosperidad-a-sus-pueblos-noticia-1172715
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/futbol/futbol-mundial/mohamed-salah-conoce-a-los-finalistas-al-balon-de-oro-africano-fotos-noticia-1172717
-------------------- End Article ----------------------
URL Article

URL Article===>https://rpp.pe/mundo/estados-unidos/donald-trump-felicita-a-todos-por-el-2019-incluso-a-a-los-criticones-y-a-los-medios-de-comunicacion-falsos-noticia-1172669
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/futbol/mas-futbol/cristiano-ronaldo-y-georgina-rodriguez-asi-despidieron-el-ano-2018-en-dubai-noticia-1172664
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/lima/actualidad/mml-desbloquea-a-usuarios-del-twitter-en-primer-dia-de-gestion-de-jorge-munoz-noticia-1172674
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/politica/gobierno/pedro-chavarry-martin-vizcarra-partio-desde-brasil-rumbo-a-peru-tras-conocer-destitucion-de-fiscales-noticia-1172663
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/cine/cartelera/spider-man-un-nuevo-universo-lee-la-critica-sobre-la-cinta-de-animacion-del-heroe-aracnido-spider-man-noticia-11

URL Article===>https://rpp.pe/politica/actualidad/glave-chavarry-se-comporta-como-defensor-de-la-senora-k-y-de-los-investigados-por-caso-lava-jato-noticia-1172614
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/politica/judiciales/congresistas-rechazan-medida-de-pedro-chavarry-sobre-rafael-vela-y-jose-domingo-perez-noticia-1172617
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/futbol/peruanos-en-el-mundo/fichajes-laliga-luis-advincula-nominado-a-mejor-fichaje-de-la-liga-santander-2018-19-noticia-1172600
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/lima/judiciales/carlos-tubino-apoya-decision-de-pedro-chavarry-vela-y-domingo-perez-son-operadores-politicos-noticia-1172606
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/peru/la-libertad/trujillanos-hacen-compras-en-medio-de-pestilencia-por-colapso-de-desague-noticia-1172605
-----------

URL Article===>https://rpp.pe/cultura/asiapop/mira-el-emocionante-primer-trailer-de-pokemon-the-movie-mewtwo-strikes-back-evolution-noticia-1172536
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/cultura/mas-cultura/meghan-markle-kate-middleton-los-principes-harry-y-william-comparten-sus-momentos-favoritos-del-2018-noticia-1172529
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/economia/economia/indecopi-autos-marca-nissan-presentarian-falla-en-la-llave-de-encendido-noticia-1172540
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/mundo/actualidad/bienvenido-2019-los-primeros-paises-del-mundo-en-recibir-el-ano-nuevo-fotos-noticia-1172478
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/lucha/wwe/wwe-njpw-lo-mejor-de-la-lucha-libre-en-2018-noticia-1172533
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/

...

URL Article===>https://rpp.pe/lima/policiales-crimenes/ministro-del-interior-policia-atacado-en-clasico-forcejeo-con-un-hincha-por-una-banderola-noticia-1147133
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/futbol/futbol-mundial/real-madrid-y-su-homenaje-a-sus-jugadores-premiados-por-la-uefa-fotos-noticia-1147131
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/futbol/descentralizado/torneo-clausura-arbitro-del-comerciantes-vs-ayacucho-fc-fue-agredido-y-partido-se-suspendio-noticia-1147130
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/famosos/celebridades/los-simpson-matt-groening-revelo-que-michael-jackson-hizo-un-cameo-en-secreto-video-noticia-1147135
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/peru/cusco/video-cusco-un-ciclista-desaparecio-tras-caer-al-rio-vilcanota-noticia-1147127
-------------------- End Article ------------

URL Article===>https://rpp.pe/futbol/futbol-mundial/esto-dijo-la-uefa-sobre-la-ausencia-de-cristiano-ronaldo-en-la-ceremonia-noticia-1147082
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/politica/judiciales/los-cuellos-blancos-del-puerto-estas-eran-las-palabras-clave-que-usaba-la-organizacion-criminal-noticia-1147000
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/mundo/estados-unidos/eeuu-corta-todos-sus-fondos-al-programa-de-la-onu-para-palestina-noticia-1147078
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/futbol/futbol-mundial/ronaldo-nazario-llego-a-un-acuerdo-para-comprar-un-club-de-espana-noticia-1147076
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/mundo/argentina/el-peso-argentino-se-desplomo-un-21-en-una-semana-de-gran-volatilidad-noticia-1147074
-------------------- End Article ----------------------
URL Article===>http

URL Article===>https://rpp.pe/mundo/estados-unidos/video-alejandro-toledo-fue-confrontado-sobre-caso-odebrecht-en-restaurante-de-eeuu-noticia-1147013
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/cultura/mas-cultura/fotos-premios-summum-estos-son-los-10-mejores-chifas-segun-el-certamen-gastronomico-noticia-1146967
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/politica/judiciales/pablo-sanchez-cree-que-chavarry-deberia-apartarse-de-conocer-el-caso-cuellos-blancos-del-puerto-noticia-1147011
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/peru/cusco/candidatos-regionales-registran-infracciones-graves-al-reglamento-de-transito-noticia-1147008
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/peru/lambayeque/clemente-flores-oviedo-debe-ser-investigado-como-lo-establece-la-ley-noticia-1146975
-------------------- End Article ---------------

URL Article===>https://rpp.pe/futbol/seleccion-peruana/seleccion-peruana-ricardo-gareca-llego-a-amsterdam-de-cara-al-amistoso-ante-holanda-noticia-1146941
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/peru/actualidad/un-limeno-pierde-en-promedio-20-dias-al-ano-atrapado-en-el-trafico-noticia-1146916
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/politica/judiciales/pedro-chavarry-transparencia-demanda-que-el-fiscal-renuncia-de-inmediato-noticia-1146897
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/politica/elecciones/humberto-lay-pide-a-los-electores-conocer-las-propuestas-y-no-basarse-en-declaraciones-populistas-noticia-1146913
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/mundo/latinoamerica/argentina-un-juez-ordeno-que-un-hombre-pase-sabados-y-domingos-en-la-carcel-hasta-pagar-pension-de-alimentos-noticia-1146928
--------------

URL Article===>https://rpp.pe/futbol/futbol-mundial/europa-league-2018-19-asi-quedo-la-fase-de-grupos-del-certamen-noticia-1146875
-------------------- End Article ----------------------
URL Article===>https://rpp.pe/futbol/futbol-mundial/antoine-griezmann-fue-elegido-el-mejor-jugador-de-la-europa-league-noticia-1146872
-------------------- End Article ----------------------
Tiempo de ejecución es 9773.014614 segundos


In [33]:
df_RPP.head()

,Fecha,Categoria,Autor,Titulo,Resumen,Contenido
0,2019-01-01 23:46:41-05:00,Asia,Redacción,Xi Jinping advierte que China no renunciará al...,"El presidente Xi Jinping dijo que China ""se re...",China no renunciará al uso de la fuerza milita...
1,2019-01-01 22:38:35-05:00,Judiciales,Redacción,Rafael Vela aseguró que no entregarán informac...,El fiscal Rafael Vela comentó que el acuerdo n...,El fiscal Rafael Vela aseguró este martes que ...
2,2019-01-01 22:34:11-05:00,Actualidad,Redacción,Ministerio de Salud presentó a los primeros be...,Los cuatro nacimientos se dieron en buenas con...,El pequeño Kallens Adriano fue el primer bebé ...
3,2019-01-01 22:30:45-05:00,Actualidad,Redacción,Detienen a dos policías y una abogada acusados...,La intervención ocurrió en Tarapoto. Un ciudad...,Dos policías de la Divincri de Tarapoto (San M...
4,2019-01-01 21:56:23-05:00,Lima,Redacción,Así informó la prensa internacional sobre la r...,La decisión del titular del Ministerio Público...,La decisión del titular del Ministerio Público...


In [34]:
df_RPP.shape

(11977, 6)

In [35]:
df_RPP.to_csv('RPP_4meses.csv',encoding='utf_8')